In [6]:
using Parameters
using LinearAlgebra
using Combinatorics
using StatsBase
using NLopt
using Random
using SparseArrays
using Optim
using TimerOutputs

using Pkg; Pkg.activate(".")
using HypergraphModularity

 Activating environment at `~/hypergraph_modularities_code/Project.toml`


In [7]:
dataset = "contact-primary-school-classes"
# dataset = "congress-bills"
# dataset = "walmart-trips"
# dataset = "TrivagoClickout"
kmax_ = 6

H, Z = read_hypergraph_data(dataset,kmax_)
α = zeros(2*kmax_);

In [8]:
kmax = maximum(keys(H.E))
kmin = minimum(keys(H.E))

n = length(H.D)

# all-or-nothing
function ω(p,α)
    k = p[2]
    δ = p[1]
    return ((1+(1-δ))*n)^α[k] / (n^α[k + kmax])
end

Ω = allOrNothingIntensityFunction(ω, kmax)

IntensityFunction(ω, HypergraphModularity.var"#7#12"(), [(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5)], HypergraphModularity.var"#9#14"(), nothing)

In [9]:
α = learnParameters(H, Z, Ω, α; n_iters = 100, amin = -50, amax = 50)

12-element Array{Float64,1}:
 -11.80339887498949
  -2.613754096785692
 -10.849115643266169
 -11.159452463383706
  -8.149804169026112
  11.80339887498948
  -0.7354134896157385
  -7.560975233879389
  -5.871720117745739
  -0.16029253215960226
 -11.80339887498949
 -11.80339887498949

In [10]:
Q = modularity(H, Z, Ω; α = α)

-213295.3444976664076524447647172272592931589442603511823641855945725690861953082

In [14]:
Z = SuperNode_PPLouvain(H, Ω; α = α, verbose = false)

10

In [13]:
@time Z_dyadic = CliqueExpansionModularity(H,5.0);
print(length(unique(Z_dyadic)))

  0.274750 seconds (858.59 k allocations: 52.520 MiB, 7.16% gc time)
11

In [14]:
α = learnParameters(H, Z_dyadic, Ω, α; n_iters = 1000, amin = -50, amax = 50)

12-element Array{Float64,1}:
 -11.80339887498949
  -2.2045642186594256
  -6.987206542392752
  -9.49503273218892
  -9.110361234933468
  11.80339887498948
  -0.29248717860487505
  -3.6000550927253436
  -4.11226135503116
  -1.2341174245816986
 -11.80339887498949
 -11.80339887498949

In [15]:
Q_dyadic = modularity(H, Z_dyadic, Ω; α = α)

-209135.1614190830599307882290221387128096005256268391737480195343589656842254897

In [11]:
cut_weights, vol_weights, e2n, n2e,w,d,elen = AON_Inputs(H,ω,α,kmax);
@assert(d == H.D)

BoundsError: BoundsError: attempt to access 12-element Array{Float64,1} at index [0]

In [13]:
## Full supernode version
verbose = true
maxits = 100
randflag = true

# Here's where you can set the warm start.

Zwarm = Z_dyadic    # can optionally specify a warm start clustering
#Zwarm = collect(1:n)  # no warm start
@time Zset = SuperNode_PPLouvain(n2e,e2n,w,d,elen,cut_weights,vol_weights,kmax,randflag,maxits,verbose,Zwarm);
Z = Zset[:,end];

clus = length(unique(Z))
println("New Version: $clus clusters")

UndefVarError: UndefVarError: n2e not defined

In [16]:
function f(x, y = length(x))
    print(x, y)
end
f(2)

21

In [335]:
α = learnParameters(H, Z, Ω, α; n_iters = 1000, amin = -50, amax = 50)
modularity(H, Z, Ω; α = α)

-6.530194075979064911921053048773569231070296312879052401225360601617792001351575e+06

In [339]:
α̂ = α
verbose = false
maxits = 100
randflag = true

println("POLYADIC")
print(rpad("iteration", 20))
print(rpad("Q", 15))
print(rpad("groups", 10))
println(rpad("time (s)", 10))
println(rpad("",  65, "-"))

Ẑ = zero(Z)
# Zwarm = collect(1:n)  # no warm start
Zwarm = Z

for i = 1:50
    cut_weights, vol_weights, e2n, n2e,w,d,elen = AON_Inputs(H,ω,α̂,kmax);
    tic = time()
    Zset = SuperNode_PPLouvain(n2e,e2n,w,d,elen,cut_weights,vol_weights,kmax,randflag,maxits,verbose,Zwarm)
    toc = time()-tic
    Ẑ = Zset[:,end]
        
    α̂ = learnParameters(H, Ẑ, Ω, α̂; n_iters = 100, amin = -50, amax = 50)
    Q = modularity(H, Ẑ, Ω; α = α̂)
    
    print(rpad("$i", 20))
    print(rpad("$(round(Q))", 15))
    print(rpad("$(length(unique(Ẑ)))", 10))
    println(rpad("$(round(toc; digits=3))", 10))
#     Zwarm = split_clusters(Ẑ)
#     Zwarm = Ẑ
end

POLYADIC
iteration           Q              groups    time (s)  
-----------------------------------------------------------------
1                   -6.51418e+06   16        3.077     
2                   -6.510683e+06  14        2.791     
3                   -6.487849e+06  16        2.296     
4                   -6.416958e+06  10        3.1       
5                   -6.400866e+06  12        2.295     
6                   -6.396077e+06  10        2.123     
7                   -6.391807e+06  13        2.396     
8                   -6.388036e+06  13        2.205     
9                   -6.384698e+06  12        2.33      
10                  -6.381769e+06  13        2.354     
11                  -6.379269e+06  13        2.393     
12                  -6.377164e+06  10        2.467     
13                  -6.375117e+06  9         2.538     
14                  -6.373548e+06  13        2.378     
15                  -6.372277e+06  12        2.455     
16                  -6.371232

InterruptException: InterruptException:

In [265]:
function split_clusters(Z)
    Z_split = zero(Z)
    zmax = maximum(Z)
    for i ∈ 1:length(Z) 
        Z_split[i] = rand() < 0.5 ? Z[i] : Z[i] + zmax
    end
    return Z_split
end

split_clusters (generic function with 1 method)